# testing RAG with sql databases

In [1]:
import os 
from pyprojroot import here
import chromadb
from openai import AzureOpenAI
import warnings
import pandas as pd

warnings.filterwarnings('ignore')
from dotenv import load_dotenv

print(f'Load environment variables:{load_dotenv()}')


Load environment variables:True


In [2]:
azure_open_api_key=os.getenv('AZURE_OPENAI_API_KEY')
azure_openai_endpoint=os.getenv('AZURE_OpenAI_ENDPOINT')
azure_openai_api_version=os.getenv('AZURE_OpenAI_API_VERSION')

In [3]:
azure_client=AzureOpenAI(
    api_key=azure_open_api_key,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint
)

chroma_client=chromadb.PersistentClient(path=str(here('data/chroma')))

# create a collection of data injection

In [4]:
# collection = chroma_client.create_collection(name='titanic_small')
collection=chroma_client.create_collection(name='titanic_small')


In [5]:
file_dir=here('data/for_upload/titanic_small.csv')
df=pd.read_csv(file_dir)

In [6]:
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35,0,0,8.0500


In [7]:
df.loc[df['Pclass']==3]

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22,1,0,7.2500
2,1,3,Miss. Laina Heikkinen,female,26,0,0,7.9250
4,0,3,Mr. William Henry Allen,male,35,0,0,8.0500
5,0,3,Mr. James Moran,male,27,0,0,8.4583
7,0,3,Master. Gosta Leonard Palsson,male,2,3,1,21.0750
8,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27,0,2,11.1333
10,1,3,Miss. Marguerite Rut Sandstrom,female,4,1,1,16.7000
12,0,3,Mr. William Henry Saundercock,male,20,0,0,8.0500
13,0,3,Mr. Anders Johan Andersson,male,39,1,5,31.2750
14,0,3,Miss. Hulda Amanda Adolfina Vestrom,female,14,0,0,7.8542


# convert the SQL data to embeddings

In [8]:
docs=[]
metadatas=[]
ids=[]
embeddings=[]

azure_client=AzureOpenAI(
    api_key=azure_open_api_key,
    api_version=azure_openai_api_version,
    azure_endpoint=os.getenv('AZURE_OPENAI_EMBEDDINGS_ENDPOINT')
)

for index,row in df.iterrows():
    output_str=''
    # treat each row as a seperate chunk
    for col in df.columns:
        output_str+=f'{col}:{row[col]},\n'
    response=azure_client.embeddings.create(
        input=output_str,
        model='text-embedding-ada-002'
    )
    embeddings.append(response.data[0].embedding)
    docs.append(output_str)
    metadatas.append({'source':'titanic_small'})
    ids.append(f'id{index}')


In [9]:
docs

['Survived:0,\nPclass:3,\nName:Mr. Owen Harris Braund,\nSex:male,\nAge:22,\nSiblings/Spouses Aboard:1,\nParents/Children Aboard:0,\nFare:7.25,\n',
 'Survived:1,\nPclass:1,\nName:Mrs. John Bradley (Florence Briggs Thayer) Cumings,\nSex:female,\nAge:38,\nSiblings/Spouses Aboard:1,\nParents/Children Aboard:0,\nFare:71.2833,\n',
 'Survived:1,\nPclass:3,\nName:Miss. Laina Heikkinen,\nSex:female,\nAge:26,\nSiblings/Spouses Aboard:0,\nParents/Children Aboard:0,\nFare:7.925,\n',
 'Survived:1,\nPclass:1,\nName:Mrs. Jacques Heath (Lily May Peel) Futrelle,\nSex:female,\nAge:35,\nSiblings/Spouses Aboard:1,\nParents/Children Aboard:0,\nFare:53.1,\n',
 'Survived:0,\nPclass:3,\nName:Mr. William Henry Allen,\nSex:male,\nAge:35,\nSiblings/Spouses Aboard:0,\nParents/Children Aboard:0,\nFare:8.05,\n',
 'Survived:0,\nPclass:3,\nName:Mr. James Moran,\nSex:male,\nAge:27,\nSiblings/Spouses Aboard:0,\nParents/Children Aboard:0,\nFare:8.4583,\n',
 'Survived:0,\nPclass:1,\nName:Mr. Timothy J McCarthy,\nSex:male

In [10]:
print(metadatas)
print(ids)

[{'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}]
['id0', 'id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9', 'id10', 'id11', 'id12', 'id13', 'id14', 'id15', 'id16', 'i

In [11]:
embeddings[0][:10]

[-0.005806858651340008,
 -0.022392014041543007,
 -0.017091887071728706,
 -0.025856956839561462,
 0.004077811725437641,
 0.036484602838754654,
 -0.014421279542148113,
 -0.0008816428016871214,
 -0.02277548611164093,
 -0.018639469519257545]

In [12]:
collection.add(
    documents=docs,
    metadatas=metadatas,
    embeddings=embeddings,
    ids=ids
)

In [13]:
print(collection.count())

30


# RAG operations

In [14]:
from openai import AzureOpenAI


In [15]:
model_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL')
azure_openai_api_key=os.getenv('AZURE_OPENAI_API_KEY')
azure_openai_endpoint=os.getenv('AZURE_OpenAI_ENDPOINT')
azure_client=AzureOpenAI(
    api_key=azure_open_api_key,
    api_version=azure_openai_api_version,
    azure_endpoint=os.getenv('AZURE_OPENAI_EMBEDDINGS_ENDPOINT')
)
llm=AzureOpenAI(
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    azure_endpoint=azure_openai_endpoint,
    api_key=os.getenv('AZURE_OPENAI_API_KEY')
)

In [16]:
query_texts='what is the maximum age of a male survivor'
response=azure_client.embeddings.create(
    input=query_texts,
    model='text-embedding-ada-002'
)
query_embeddings=response.data[0].embedding


# load the chromaDB collection for vector search

In [17]:
vectordb=chroma_client.get_collection(name='titanic_small')

In [18]:
result=vectordb.query(
    query_embeddings=query_embeddings,
    n_results=1
)
result

{'ids': [['id23']],
 'distances': [[0.47334641130996247]],
 'metadatas': [[{'source': 'titanic_small'}]],
 'embeddings': None,
 'documents': [['Survived:1,\nPclass:1,\nName:Mr. William Thompson Sloper,\nSex:male,\nAge:28,\nSiblings/Spouses Aboard:0,\nParents/Children Aboard:0,\nFare:35.5,\n']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [19]:
system_role = "You will recieve the user's question along with the search results of that question over a database. Give the user the proper answer."
prompt = f"User's question: {query_texts} \n\n Search results:\n {result}"

message=[
    {
        'role':'system','content':str(system_role)
    },
    {
        'role':'user','content':prompt
    }
]

In [20]:
response=llm.chat.completions.create(
    model=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    messages=message
)


In [21]:
response.choices[0].message.content

'The maximum age of a male survivor in the given database is 28.'

# fact check

- below response is totally incorrect
- therefore we choose SQL agent for this scenario

In [28]:
print(df.loc[(df['Survived']==1) & (df['Age']==max(df['Age'] ) )])
#& df['Sex']=='male'

    Survived  Pclass                     Name     Sex  Age  \
11         1       1  Miss. Elizabeth Bonnell  female   58   

    Siblings/Spouses Aboard  Parents/Children Aboard   Fare  
11                        0                        0  26.55  
